In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

# read 데이터
dly = pd.read_csv('AFSNT_DLY.CSV', encoding="cp949")
dly.head()


,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,NaN,NaN
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,NaN,NaN
2,2019,9,16,월,ARP1,ARP3,L,L1720,A,14:40,NaN,NaN
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,NaN,NaN
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,NaN,NaN


In [3]:
dly.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,NaN,NaN
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,NaN,NaN
2,2019,9,16,월,ARP1,ARP3,L,L1720,A,14:40,NaN,NaN
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,NaN,NaN
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,NaN,NaN


In [4]:
dly['hour'] = pd.to_datetime(dly['STT'],format= '%H:%M').dt.hour

In [5]:
import urllib
import json
import pandas as pd
import dateutil.parser
#날씨데이터 json 읽어오는 함수

#나중에 9월 1일을 9월 16일로 바꾼 후 실행..
#(읽어오는거 확인용으로 9월 1일 부터...)
def readJSON(area):
    
    request = urllib.request.urlopen('https://api.aerisapi.com/forecasts/'+area+',korea?from=09/16/2019&format=json&filter=1hr&limit=999&client_id=gHOhinKWCL1fwDUpI1Ec7&client_secret=TRamBTyXpORXcmDKVqc2S4i4mnCjVxxHMt6cllui')
    response = request.read()
    data= json.loads(response)


    if data['success']:
        return data

    else:
        print("An error occurred: %s" % (data['error']['description']))
        return ""
        request.close()

In [6]:
data=readJSON("jeju")
data

{'success': True,
 'error': None,
 'response': [{'loc': {'long': 126.522, 'lat': 33.51},
   'interval': '1hr',
   'periods': [{'timestamp': 1568559600,
     'validTime': '2019-09-16T00:00:00+09:00',
     'dateTimeISO': '2019-09-16T00:00:00+09:00',
     'maxTempC': 22,
     'maxTempF': 71,
     'minTempC': 22,
     'minTempF': 71,
     'avgTempC': 22,
     'avgTempF': 71,
     'tempC': 22,
     'tempF': 71,
     'pop': 0,
     'precipMM': 0,
     'precipIN': 0,
     'iceaccum': None,
     'iceaccumMM': None,
     'iceaccumIN': None,
     'maxHumidity': 82,
     'minHumidity': 82,
     'humidity': 82,
     'uvi': 0,
     'pressureMB': 1012,
     'pressureIN': 29.88,
     'sky': 4,
     'snowCM': 0,
     'snowIN': 0,
     'feelslikeC': 22,
     'feelslikeF': 72,
     'minFeelslikeC': 22,
     'minFeelslikeF': 72,
     'maxFeelslikeC': 22,
     'maxFeelslikeF': 72,
     'avgFeelslikeC': 22,
     'avgFeelslikeF': 72,
     'dewpointC': 19,
     'dewpointF': 66,
     'maxDewpointC': 19,
     

In [7]:
#날씨 csv 저장
def storeWeather():
    weather=pd.DataFrame(columns=['SDT_YY', 'SDT_MM', 'SDT_DD','hour','ARP','temp', 'hum', 'dew', 'windSpeed','hpa'])

    for i in range(15):
        area=["seoul","busan","jeju","daegu","ulsan","cheongju","muan","gwangju","yeosu"
              ,"yangyang","pohang","sacheon","gunsan","wonju","incheon"]
        d=readJSON(area[i])
        response=d['response'][0]
        data=response['periods']
    
    
        for j in range(len(data)):
            temp=data[j]
            date=dateutil.parser.parse(temp['dateTimeISO'])
            temp2=pd.Series([date.year,date.month,date.day,date.hour,"ARP"+str(i+1),temp['tempC'],
                         temp['humidity'],temp['dewpointC'],temp['windGustKTS'],temp['pressureMB']],
                        index=['SDT_YY', 'SDT_MM', 'SDT_DD','hour','ARP','temp', 'hum', 'dew', 'windSpeed','hpa'])
        
            weather=weather.append(temp2,ignore_index=True)
    
    #fog 모델하고의 단위 맞추는 작업
    weather['windSpeed']=weather['windSpeed'].astype("float")*1852/3600
    
    weather.to_csv("newWeather.csv",index=False)

In [8]:
#미래 날씨 데이터 합치기

from datetime import datetime
now=datetime.now()
if now.day>=16:
    we=pd.read_csv('newWeather.csv', encoding="cp949")
    
else:
    print(now.day)
    storeWeather()
    we=pd.read_csv('newWeather.csv', encoding="cp949")

8


In [9]:
final= pd.merge(we, dly, on=['SDT_YY', 'SDT_MM', 'SDT_DD', 'ARP','hour'])
final

#---------------------------------------

,SDT_YY,SDT_MM,SDT_DD,hour,ARP,temp,hum,dew,windSpeed,hpa,SDT_DY,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,L,L1701,D,6:40,NaN,NaN
1,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP12,J,J1634,D,6:50,NaN,NaN
2,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,F,F1201,D,6:10,NaN,NaN
3,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,F,F1203,D,6:15,NaN,NaN
4,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,J,J1204,D,6:20,NaN,NaN
5,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,A,A1901,D,6:05,NaN,NaN
6,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,H,H1151,D,6:15,NaN,NaN
7,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,F,F1205,D,6:50,NaN,NaN
8,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,A,A1903,D,6:25,NaN,NaN
9,2019,9,16,6,ARP1,22,69,16,1.028889,1015,월,ARP3,H,H1101,D,6:25,NaN,NaN


In [10]:
# 파생변수
final['ARPODP'] = final['ARP'] + '_' + final['ODP']

In [11]:
# 필요없는 column 제거
final.drop(columns=['FLT', 'DLY', 'DLY_RATE'], axis=1, inplace=True)


In [12]:
# 시간 + 분 합침.
final['time']= (pd.to_datetime(final['STT'],format= '%H:%M').dt.hour).astype(str) + (pd.to_datetime(final['STT'],format= '%H:%M').dt.minute).astype(str)

#dly 스케일링
from sklearn.preprocessing import MinMaxScaler
final['time']= final['time'].astype('int')
scaler = MinMaxScaler()
final[['time']] = scaler.fit_transform(final[['time']])


#예정시각 삭제
final.drop(['STT'], axis = 1, inplace=True)


In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

for i in range(5):
    day= dly[dly['SDT_DD']==26+i]
    
    temps=[]
    hums=[]
    dews=[]
    winds=[]

    for j in range(len(day)):
        prev_5= we[ (we['ARP']==day.iloc[j]['ARP']) & (we['hour']==day.iloc[j]['hour']) & 
                      (we['SDT_DD'] >= 26+i-5) & (we['SDT_DD'] < 26+i) ]        
        print('prevs', prev_5)
        temp=0.0
        hum=0.0
        dew=0.0
        windSpeed=0.0

        for k in range(len(prev_5)):
            temp += prev_5.iloc[k]['temp']
            hum += prev_5.iloc[k]['hum']
            dew += prev_5.iloc[k]['dew']
            windSpeed += prev_5.iloc[k]['windSpeed']
        
        print(temp, hum, dew, windSpeed)
        
        if temp > 0:
            temp= temp / len(prev_5)
        if hum > 0:
            hum= hum / len(prev_5)
        if dew > 0:
            dew= dew / len(prev_5)
        if windSpeed > 0:
            windSpeed= windSpeed / len(prev_5)
        
        temps.append(temp)
        hums.append(hum)
        dews.append(dews)
        winds.append(windSpeed)

    
#    print(len(temps), len(hums), len(dews), len(winds))

    day['temp']= temps
    day['hum']=hums
    day['dew']=dews
    day['windSpeed']=winds

    print(day)

    final= final.append(day)
  

In [ ]:
print(final.tail())
final[final['SDT_DD']==26]

In [ ]:
from sklearn import preprocessing
# DLY도 1과 0으로 데이터 처리
le = preprocessing.LabelEncoder()
final[['ARP']] = le.fit_transform(final[['ARP']])
final[['ODP']] = le.fit_transform(final[['ODP']])
final[['FLO']] = le.fit_transform(final[['FLO']])
final[['ARPODP']] = le.fit_transform(final[['ARPODP']])

In [ ]:
## Todo: 요일은 계속 원핫???
one_hot_dy = pd.get_dummies(final['SDT_DY'])
final = final.drop(['SDT_DY'],axis = 1)
final = final.join(one_hot_dy)
final.rename(columns={"일":"Sun","월":"Mon","화":"Tue","수":"Wed","목":"Thu","금":"Fri","토":"SAT","일":"Sun"                    
                  }, inplace=True)

In [ ]:
# YY랑 DD삭제
final.drop(columns=['SDT_YY', 'SDT_DD'], axis=1, inplace=True)

In [ ]:
final.columns

final.drop(columns=['hour'], axis=1, inplace=True)

# 중요도 낮은 변수들 제거 

In [ ]:
from sklearn.preprocessing import RobustScaler
import pickle
from sklearn.externals import joblib

def fogModel(df):
    # 날씨 missing 값들은 0으로 대체
    df.fillna(0, inplace = True)
    
    # 모델에서 쓰인 Scaling기법 적용
    scaler = RobustScaler()
    df[['hum', 'dew','temp','windSpeed']] = scaler.fit_transform(df[['hum', 'dew','temp','windSpeed']])
    
    # 저장된 모델 불러오기
    clf_from_joblib = joblib.load('fogmodel.pkl') 

    # 지연 율 저장
    fog_prob = clf_from_joblib.predict_proba(df)
    

    fog_column = []
    # dly_rate에 지연율 저장
    for i in fog_prob:
        fog_column.append(i[1])
        
    
    return fog_column

In [ ]:
# df에서 fog관련 column만 함수에 넘김
fog = final[["temp","hum","dew","windSpeed"]]

fog_column = fogModel(fog)

In [ ]:
# 안개 column 추가
final['fog'] = fog_column

# 안개 관련 column 제거
final.drop(columns=['hum', 'dew','temp','windSpeed'], axis=1, inplace=True)
final.head()

In [ ]:
# 도착, 출발 데이터 분리
final_A = final[final['AOD']=='A']
final_D = final[final['AOD']=='D']

# AOD column삭제 
final_A = final_A.drop(['AOD'],axis = 1)
final_D = final_D.drop(['AOD'],axis = 1)

In [ ]:
(final_A.columns)

In [ ]:
# ------------------------------ #

In [ ]:
########## 아직 저장된 모델이 없음

# df_A 모델 -----------------------------------------------

# df_A 모델 불러오기
predict_dealy_A_joblib = joblib.load('predict_delay_A.pkl') 

# DLY 저장하기
dly_A = predict_dealy_A_joblib.predict(final_A)

# DLY_RATE 저장하기
dly_A_prob = predict_dealy_A_joblib.predict_proba(final_A)

dly_rate_A = []

# dly_rate에 지연율 저장
for i in dly_A_prob:
    dly_rate_A.append(i[1])
    
# DateFrame에 DLY, DLY_RATE추가
final.loc[dly['AOD']=='A','DLY'] = dly_A
final.loc[dly['AOD']=='A','DLY_RATE'] = dly_rate_A

# ---------------------------------------------------------

In [ ]:
# df_D 모델 -----------------------------------------------

# df_D 모델 불러오기
predict_dealy_D_joblib = joblib.load('predict_delay_D.pkl') 

# DLY 저장하기
dly_D = predict_dealy_D_joblib.predict(df_D)

# DLY_RATE 저장하기
dly_A_prob = predict_dealy_A_joblib.predict_proba(df_D)

dly_rate_D = []

# dly_rate에 지연율 저장
for i in dly_D_prob:
    dly_rate_D.append(i[1])
    
# DateFrame에 DLY, DLY_RATE추가
final.loc[dly['AOD']=='D','DLY'] = dly_D
final.loc[dly['AOD']=='D','DLY_RATE'] = dly_rate_D

# ---------------------------------------------------------

In [ ]:
# 라벨을 1,0에서 -> Y,N으로 
final.loc[dly['DLY']==1,'DLY'] = 'Y'
final.loc[dly['DLY']==0,'DLY'] = 'N'